##### Copyright 2018 The TensorFlow Authors. 

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classificazione di testo con testo pre-elaborato: Recensioni di film

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/it/tutorials/keras/text_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Note: La nostra comunità di Tensorflow ha tradotto questi documenti. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Questo notebook classifica recensioni di film come *positive* o *negative* usando il testo delle revisioni. Questo è un esempio di classificazione *binaria*—o a due classi, un importante tipo di problema di machine learning largamente applicabile.

Useremo il [dataset IMDB](https://www.tensorflow.org/datasets/catalog/imdb_reviews) che contiene il testo di 50.000 recensioni di film dall'[Internet Movie Database](https://www.imdb.com/). Esse sono divise in 25,000 recensioni per l'addestramento e 25,000 revisioni per la verifica. Gli insiemi di addestramento e verifica sono *bilanciati*, nel senso che essi contengono un eguale numero di recensioni positive e negative.

Questo notebook usa [tf.keras](https://www.tensorflow.org/guide/keras), una API di alto livello per costruire e addestrare modelli in TensorFlow. Per un tutorial più avanzato di classificazione del testo che usa `tf.keras`, vedere la [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/).

## Setup

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

In [ ]:
from tensorflow import keras

!pip install tensorflow-datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

import numpy as np

print(tf.__version__)

<a id="download"></a>

## Scarichiamo il dataset IMDB

Il dataset di recensioni di film IMDB viene compattato in `tfds`. Esso è stato già pre-elaborato in modo che le recensioni (sequenze di parole) sono state convertite in sequenze di interi, ove ciascun intero rappresenta una particolare parola in un vocabolario.

Il codice che segue scarica il dataset IMDB sulla vostra macchina (o usa una copia locale se lo avete scaricato in precedenza):

Per codificare il vostro testo vedere il [tutorial sul caricamento di testo](../load_data/text.ipynb)

In [ ]:
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

<a id="encoder"></a>

## Proviamo il codificatore

 Il dataset `info` include il codificatore di testo (un `tfds.features.text.SubwordTextEncoder`).

In [ ]:
encoder = info.features['text'].encoder

In [ ]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))

Questo codificatore di testo codifica reversibilmente ogni stringa:

In [ ]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

assert original_string == sample_string

Il codificatore codifica la stringa spezzandola in sotto-parole o caratteri se la parola non è presente nel suo vocabolario. In questo modo, più una stringa somiglia al dataset, più corta sarà la rappresentazione codificata.

In [ ]:
for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

## Esploriamo i dati

Prendiamoci un momento per capire il formato dei dati. Il dataset è pre-elaborato: ogni esempio è un vettore di interi che rappresenta le parole della recensione del film. 

I testi delle recensioni sono stati convertiti in interi, dove ciascun intero rappresenta un particolare frammento di parola nel vocabolario. 

Ogni etichetta è un valore intero tra 0 e 1, dove 0 è una recensione negativa, e 1 una recensione positiva.

Qui ciò a cui somiglia la prima recensione:

In [ ]:
for train_example, train_label in train_data.take(1):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

La struttura `info` contiene il codificatore/decodificatore. Il decodificatore può essere usato per recuperare il testo originale:

In [ ]:
encoder.decode(train_example)

## Prepariamo i dati per l'addestramento

Vorrete creare lotti di dati di addestramento per il vostro modello. Le recensioni sono tutte di lunghezza diversa, così usiamo `padded_batch` per riempire di zeri le sequenze durante la suddivisione in lotti:

In [ ]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_data
    .padded_batch(32))

Ogni lotto avrà una forma del tipo `(batch_size, sequence_length)` e dato che il riempimento è dinamico, ogni lotto avrà una lunghezza diversa:

In [ ]:
for example_batch, label_batch in train_batches.take(2):
  print("Batch shape:", example_batch.shape)
  print("label shape:", label_batch.shape)
  

## Costruiamo il modello

La rete neurale viene creata impilando livelli—ciò richiede due decisioni architetturali principali:

* Quanti livelli usare nel modello?
* Quante *unità nascoste* usare in ciascun livello?

In questo esempio, i dati di input sono costituiti da un vettore di parole-indici. Le etichette da prevedere sono 0 oppure 1. Costruiamo un modello in stile "Continuous bag-of-words" per questo problema:

Attenzione: Questo modello non usa la mascheratura, così il riempimento di zeri viene utilizzato come parte dell'input, così la lunghezza del riempimento può influire sull'output.  Per evitare ciò, vedere la [guida al riempimento e mascheramento](../../guide/keras/masking_and_padding).

In [ ]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1)])

model.summary()

I livelli sono impilati in sequenza per implementare il classificatore:

1. Il primo livello è un livello `Incorporamento`. Questo livello prende il vocabolario codificato in interi e guarda il vettore di incorporamento per ogni parola-indice. Questi vettori sono assimilati durante l'addestramento del modello. I vettori aggiungono una dimensione al vettore di output. Le dimensioni risultanti sono: `(batch, sequence, embedding)`.
2. Successivamente, un livello `GlobalAveragePooling1D` restituisce in output un vettore di lunghezza fissa per ogni esempio mediando sulle dimensioni della sequenza. Ciò permette al modello di gestire input di lunghezza variabile, nel modo più semplice possibile.
3. Questo vettore di output a lunghezza fissa viene passato attraverso un livello completamente connesso (`Denso`) con 16 unità nascoste.
4. L'ultimo livello è connesso densamente ed ha un solo nodo di output. Usando la funzione di attivazione `sigmoid`, questo valore è un decimale tra 0 e 1, che rappresenta una probabilità, o un livello di confidenza.

### Unità nascoste

Il modello di cui sopra ha due livelli intermedi o "nascosti", tra l'input e l'output. Il numero di output (unità, nodi o neuroni) è la dimensione dello spazio di rappresentazione del livello. In altre parole, l'ammontare della libertà di cui dispone la rete quando durante l'apprendimento di una rappresentazione interna.

Se un modello ha più di un'unità nascosta (uno spazio di rappresentazione dimensionale più grande), e/o più livelli, allora la rete può apprendere rappresentazioni più complesse. Comunque, ciò rende la rete computazionalmente più costosa e può condurre all'apprendimento di pattern indesiderati—pattern che aumentano le prestazioni sui dati di addestramento ma non sui dati di test. Questo (fenomeno n.d.r.) viene chiamato *overfitting* (sovradattamento n.d.t.), e verrà esplorato in seguito.

### Funzione obiettivo e ottimizzatore

Un modello, per l'addestramento, ha bisogno di una funzione obiettivo e di un ottimizzatore. Essendo questo un problema di classificazione binaria e l'output del modello una probabilità (un livello a unità singola con un'attivazione sigmoid), useremo la funzione obiettivo `binary_crossentropy`.

Questa non è l'unica scelta possibile per una funzione obiettivo, potreste, per esempio, scegliere la `mean_squared_error`. In generale, però, `binary_crossentropy` è migliore per gestire probabilità—essa misura la "distanza" tra distribuzioni di probabilità o, nel nostro caso, tra la distribuzione dei dati reali e le previsioni.

Nel seguito, quando esploreremo i problemi di regressione (diciamo, per prevedere il prezzo di una casa), vedremo come usare un'altra funzione obiettivo chiamata scarto quadratico medio.

Adesso, configuriamo il modello per usare un ottimizzatore ed una funzione obiettivo:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Addestriamo il modello

Addestrare il modello passando l'oggetto `Dataset` alla funzione di allenamento del modello. Impostare il numero di epoche.

In [ ]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

## Valutiamo il modello

E andiamo a vedere come si comporta il modello. Saranno restituiti due valori. loss (Perdita n.d.t.) (un numero che rappresenta il nostro errore, per cui valori piccoli sono migliori), e accuracy (accuratezza n.d.t).

In [ ]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Questo approccio abbastanza ingenuo raggiunge un'accuratezza di circa l'87%. Con approcci più avanzati, il modello potrebbe avvicinarsi al 95%.

## Creiamo un grafico di accuratezza e obiettivo nel tempo

`model.fit()` restituisce un oggetto `History` che contiene un registro con tutto ciò che è accaduto durante l'addestramento:

In [ ]:
history_dict = history.history
history_dict.keys()

Ci sono quattro sezioni: una per ogni metrica monitorata durante l'addestramento e la validazione. Possiamo usare queste per tracciare il confronto tra l'obiettivo in addestramento e in validazione, così come l'accuratezza in addestramento e validazione:

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

In questo grafico, i punti rappresentano la perita e l'accuratezza in addestramento, mentre le linee continue sono l'obiettivo e l'accuratezza in validazione.

Notate che l'obiettivo in addestramento *decresce* con le epoche e l'accuratezza *cresce* con le epoche. Questo è quello che ci si attende quando si usa un'ottimizzazione a gradiente discendente—esso dovrebbe minimizzare la quantità obiettivo ad ogni iterazione.

Questo non accade per obiettivo e accuratezza in validazione—esse sembrano avere un picco dopo circa venti epoche. Questo è un esempio di sovradattamento: il modello ha prestazioni migliori sui dati di addestramento che su dati che non ha mai visto prima. Dopo questo punto, il modello sovra-ottimizza ed impara rappresentazioni *specifiche* dei dati di addestramento che non *generalizzano* sui dati di test.

Per questo caso particolare, non possiamo prevenire il sovradattamento fermando semplicemente l'addestramento dopo più o meno venti epoche. Nel seguito, vedremo come farlo automaticamente con una callback.